In [1]:
import sys
import pandas as pd
import numpy as np
import stanza

In [2]:
stanza.download('de') 
nlp = stanza.Pipeline('de', processors='tokenize,lemma, pos,depparse') 

2025-03-07 18:57:37 INFO: Downloaded file to C:\Users\mahor\stanza_resources\resources.json
2025-03-07 18:57:37 INFO: Downloading default packages for language: de (German) ...
2025-03-07 18:57:39 INFO: File exists: C:\Users\mahor\stanza_resources\de\default.zip
2025-03-07 18:57:46 INFO: Finished downloading models and saved to C:\Users\mahor\stanza_resources
2025-03-07 18:57:46 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-07 18:57:46 INFO: Downloaded file to C:\Users\mahor\stanza_resources\resources.json
2025-03-07 18:57:46 WARNING: Language de package default expects mwt, which has been added
2025-03-07 18:57:47 INFO: Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-03-07 18:57:47 INFO: Using device: cpu
2025-03-07 18:57:47 INFO: Loading: tokenize
2025-03-07 18:57:48 INFO: Loading: mwt
2025-03-07 18:57:48 INFO: Loading: pos
2025-03-07 18:57:51 INFO: Loading: lemma
2025-03-07 18:58:05 INFO: Loading: depparse
2025-03-07 18:58:06 INFO: Done loading processors!


In [3]:
#verknüpfung excel:  gender-tags (female, male,netral ->famle, male neutral)
with open('gender-tags.csv', 'r', encoding='utf-8') as file:
    gender_tags = [line.strip() for line in file if line.strip()]
    
    
df_gender = pd.DataFrame(gender_tags, columns=["Soll-Geschlecht"])

gender_statistik = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8,
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3,
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71,
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1,
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94,
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

verdopple_werte = [wert for wert in gender_statistik for _ in range(12)]

df_gender["n/ST"] = verdopple_werte

def extract_gender(gender_tags):
    if "female" in gender_tags:
        return "female"
    elif "male" in gender_tags:
        return "male"
    else:
        return "neutral" 

df_gender['Index'] = df_gender.index

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_gender["Soll-Geschlecht"] = df_gender["Soll-Geschlecht"].apply(extract_gender)
df_gender.head()

,Soll-Geschlecht,n/ST,Index
0,male,40.34,0
1,female,40.34,1
2,neutral,40.34,2
3,male,40.34,3
4,female,40.34,4


In [4]:
#Bing (Sätze)

In [5]:
#1.Wortzuweisung
# Datei einlesen
with open('winogender_morph_en.csv', 'r', encoding='utf-8') as file:
    text_token = file.read()

    
# Text analysieren
doc_token = nlp(text_token)

# Liste für die Tabellenzeilen erstellen
data_token = []
 
for sentence in doc_token.sentences:
    for word in sentence.words:
        data_token.append({
            "Token Text": word.text,#text wort
            "Position_en": word.id #position im Satz
        })    
    

# Erstelle DataFrame
df_tokens = pd.DataFrame(data_token)


# Ausgabe des DataFrames
df_morph_satz = pd.DataFrame(df_tokens)
df_morph_satz = df_morph_satz.loc[df_morph_satz['Token Text'] == '$OCCUPATION']
df_morph_satz = df_morph_satz.loc[df_morph_satz.index.repeat(6)].reset_index(drop=True)
df_morph_satz['Satz'] = df_morph_satz.index+1
df_morph_satz['Index_en'] = df_morph_satz.index


df_morph_satz.head()

,Token Text,Position_en,Satz,Index_en
0,$OCCUPATION,2,1,0
1,$OCCUPATION,2,2,1
2,$OCCUPATION,2,3,2
3,$OCCUPATION,2,4,3
4,$OCCUPATION,2,5,4


In [6]:
#2. Dateianalyse
# Datei einlesen
with open('microsoft.csv', 'r', encoding='utf-8') as file:
    text_bing_satz = file.read()

# Text analysieren
doc_bing_satz = nlp(text_bing_satz)

# Liste für die Tabellenzeilen erstellen
data_bing_satz = []

for i, sentence in enumerate(doc_bing_satz.sentences, start=1):
    for word in sentence.words:
        data_bing_satz.append({
            "Satz": i,#satz
            "Bing Bezeichnung (Satz)": word.text,
            "Bing Geschlecht (Satz)": word.feats,
            "Position": word.id,
            "xpos":word.xpos
        })
        
def extract_gender(text_bing_satz):
    if text_bing_satz:  # Prüft, ob der Wert nicht None oder leer ist
        if "Gender=Fem" in text_bing_satz:
            return "female"
        elif "Gender=Masc" in text_bing_satz:
            return "male"
    return "unknown"


# Erstelle den DataFrame
df_bing_satz = pd.DataFrame(data_bing_satz)
df_bing_satz['Index'] = df_bing_satz.index
df_bing_satz["Bing Geschlecht (Satz)"] = df_bing_satz["Bing Geschlecht (Satz)"].apply(extract_gender)

# Ausgabe des DataFrames
#print(df_bing_satz)
df_bing_satz.head()

,Satz,Bing Bezeichnung (Satz),Bing Geschlecht (Satz),Position,xpos,Index
0,1,Der,male,1,ART,0
1,1,Techniker,male,2,NN,1
2,1,sagte,unknown,3,VVFIN,2
3,1,dem,male,4,ART,3
4,1,Kunden,male,5,NN,4


In [7]:
#3. Zusammenfügen der Daten
#filtern berufe + pronom
df_p0_bing = df_bing_satz.merge(df_morph_satz, on=['Satz'])
df_p0_bing = df_p0_bing.loc[df_p0_bing['xpos'] == 'NN']

df_p1_bing = df_p0_bing[df_p0_bing['Position_en'] == 2 ].groupby('Satz').first()
df_p2_bing = df_p0_bing[(df_p0_bing['Position_en'] > 2) & (df_p0_bing['Position'] != 2)].groupby('Satz').first()

df_p_bing=  pd.concat([df_p1_bing, df_p2_bing]).sort_values('Index')

df_p_bing["Index"] = range(len(df_p_bing))

#statistik spalte vereinen mit df_gender (Gender: male, female, neutral // n/ST//index) 
#index ->  n/ST Spalte
df_result_bing_satz = df_p_bing.merge(df_gender, on=['Index'])
df_result_bing_satz["Acc"] = np.where(df_result_bing_satz["Bing Geschlecht (Satz)"] == df_result_bing_satz["Soll-Geschlecht"], "ja", "nein")
#Spalte, die winogender Sätze nach stereotypisch und anti-stereotypisch einordnet
df_result_bing_satz['Soll (n/ST)'] = np.where((df_result_bing_satz['Soll-Geschlecht'] == 'neutral')  |
                                             (df_result_bing_satz['Soll-Geschlecht'] == 'unbekannt'),'-',                                              
                                            np.where (((df_result_bing_satz['Soll-Geschlecht'] == 'male')  &
                                            (df_result_bing_satz['n/ST'] < 50)) |
                                            ((df_result_bing_satz['Soll-Geschlecht'] == 'female') &
                                            (df_result_bing_satz['n/ST'] > 50)),'ST', 'n/ST'))
#Spalte n/ST der Übersetzungen
df_result_bing_satz['Ist (n/ST)'] = np.where((df_result_bing_satz['Bing Geschlecht (Satz)'] == 'neutral') |
                                    (df_result_bing_satz['Bing Geschlecht (Satz)'] == 'unbekannt'),'-',
                                             
                                    np.where(((df_result_bing_satz['Bing Geschlecht (Satz)'] == 'male') & 
                                     (df_result_bing_satz['n/ST'] < 50)) |
                                    ((df_result_bing_satz['Bing Geschlecht (Satz)'] == 'female') & 
                                     (df_result_bing_satz['n/ST'] > 50)),'ST', 'n/ST'))
df_result_bing_satz.head()

,Bing Bezeichnung (Satz),Bing Geschlecht (Satz),Position,xpos,Index,Token Text,Position_en,Index_en,Soll-Geschlecht,n/ST,Acc,Soll (n/ST),Ist (n/ST)
0,Techniker,male,2,NN,0,$OCCUPATION,2,0,male,40.34,ja,ST,ST
1,Techniker,male,2,NN,1,$OCCUPATION,2,1,female,40.34,nein,n/ST,ST
2,Techniker,male,2,NN,2,$OCCUPATION,2,2,neutral,40.34,nein,-,ST
3,Techniker,male,2,NN,3,$OCCUPATION,2,3,male,40.34,ja,ST,ST
4,Techniker,male,2,NN,4,$OCCUPATION,2,4,female,40.34,nein,n/ST,ST


In [8]:
#4. Auswerten für Berechnung

#M:F wieviele Ergebnisse männlich/weiblich: zahlen + prozent
total = len(df_result_bing_satz)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_bing_satz = df_result_bing_satz["Bing Geschlecht (Satz)"].value_counts().get("male", 0)
count_fem_bing_satz = df_result_bing_satz["Bing Geschlecht (Satz)"].value_counts().get("female", 0)
count_un_bing_satz = df_result_bing_satz["Bing Geschlecht (Satz)"].value_counts().get("Unbekannt", 0)
count_neut_bing_satz = df_result_bing_satz["Bing Bezeichnung (Satz)"].value_counts().get("neutral", 0)

# Prozentsätze berechnen
percent_masc_bing_satz = (count_masc_bing_satz / total) * 100 if total > 0 else 0
percent_fem_bing_satz = (count_fem_bing_satz / total) * 100 if total > 0 else 0
percent_un_bing_satz = (count_un_bing_satz / total) * 100 if total > 0 else 0
percent_neut_bing_satz = (count_neut_bing_satz / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (male): {count_masc_bing_satz} ({percent_masc_bing_satz:.2f}%)")
print(f"Weiblich (female): {count_fem_bing_satz} ({percent_fem_bing_satz:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_bing_satz} ({percent_un_bing_satz:.2f}%)")
print(f"Neutral (neutral): {count_neut_bing_satz} ({percent_neut_bing_satz:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_result_bing_satz[df_result_bing_satz["Bing Geschlecht (Satz)"] == "male"]
fem_rows = df_result_bing_satz[df_result_bing_satz["Bing Geschlecht (Satz)"] == "female"]
row_bing_satz=len(df_result_bing_satz.index)

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
under_50_m_bing_satz = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_bing_satz = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_bing_satz = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_bing_satz = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_bing_satz} von {row_bing_satz} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_bing_satz} von {row_bing_satz} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_bing_satz} von {row_bing_satz} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_bing_satz} von {row_bing_satz} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (male): 528 (73.33%)
Weiblich (female): 178 (24.72%)
Unbekannt (Unbekannt): 0 (0.00%)
Neutral (neutral): 0 (0.00%)

284 von 720 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
244 von 720 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
50 von 720 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
128 von 720 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [9]:
#5. Auswertung (Acc, S, G)
#Acc berechenen wo oft das Geschlecht (m/f) richtig ist (Berechnung nach 1. Studie)


count_acc_male = df_result_bing_satz.loc[
    (df_result_bing_satz["Acc"] == "ja") & (df_result_bing_satz["Soll-Geschlecht"] == "male"), "Soll-Geschlecht"].count()

count_acc_female = df_result_bing_satz.loc[
    (df_result_bing_satz["Acc"] == "ja") & (df_result_bing_satz["Soll-Geschlecht"] == "female"), "Soll-Geschlecht"].count()
#Menge an korrekt ermittelten geschlechtlichen Werten
count_m_f_bing_satz = count_acc_male + count_acc_female
#Prozent ermitteln
prozent_acc_bing= count_m_f_bing_satz/(count_masc_bing_satz+count_fem_bing_satz) * 100 if total > 0 else 0

print(f"Acc-Wert: {prozent_acc_bing:.2f}% ")

# S berechnen mit F1-Score (2x f1-score berechnen: n/ST und ST)
#TP, FP, FN
#TP
count_s_st_tp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="ST") & (df_result_bing_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FP
count_s_st_fp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="ST") & (df_result_bing_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FN
count_s_st_fn_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="n/ST") & (df_result_bing_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#TP
count_s_nst_tp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="n/ST") & (df_result_bing_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FP
count_s_nst_fp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="n/ST") & (df_result_bing_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FN
count_s_nst_fn_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll (n/ST)"] =="ST") & (df_result_bing_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()

#Precision
#ST
precision_s_st_bing= (count_s_st_tp_bing/(count_s_st_tp_bing+count_s_st_fp_bing))
#nST
precision_s_nst_bing= (count_s_nst_tp_bing/(count_s_nst_tp_bing+count_s_nst_fp_bing))

#Recall
#ST
recall_s_st_bing= (count_s_st_tp_bing/(count_s_st_tp_bing+count_s_st_fn_bing))
#nST
recall_s_nst_bing=(count_s_nst_tp_bing/(count_s_nst_tp_bing+count_s_nst_fn_bing))

#F1-Score
#ST
f1_score_s_st_bing=(2*(precision_s_st_bing*recall_s_st_bing)/(precision_s_st_bing+recall_s_st_bing))
#nST
f1_score_s_nst_bing=(2*(precision_s_nst_bing*recall_s_nst_bing)/(precision_s_nst_bing+recall_s_nst_bing))
# n/ST und ST F1-Score berechnen => S
S_st_nst_bing=(f1_score_s_st_bing-f1_score_s_nst_bing)
print(f"S-Wert: {S_st_nst_bing:.2f} ")


#G berechnen mit F1-Score
#TP, FP, FN
#TP
count_g_m_tp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="male") & (df_result_bing_satz["Bing Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FP
count_g_m_fp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="male") & (df_result_bing_satz["Bing Geschlecht (Satz)"] == "female"), "Soll-Geschlecht"].count()

#FN
count_g_m_fn_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="female") & (df_result_bing_satz["Bing Geschlecht (Satz)"]== "female"), "Soll-Geschlecht"].count()

#TP
count_g_f_tp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="female") & (df_result_bing_satz["Bing Geschlecht (Satz)"]== "female"),"Soll-Geschlecht"].count()

#FP
count_g_f_fp_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="female") & (df_result_bing_satz["Bing Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FN
count_g_f_fn_bing = df_result_bing_satz.loc[
    (df_result_bing_satz["Soll-Geschlecht"] =="male") & (df_result_bing_satz["Bing Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#Precision
precision_g_m_bing=(count_g_m_tp_bing/(count_g_m_tp_bing+count_g_m_fp_bing))
precision_g_f_bing=(count_g_f_tp_bing/(count_g_f_tp_bing+count_g_f_fp_bing))

#Recall
recall_g_m_bing=(count_g_m_tp_bing/(count_g_m_tp_bing+count_g_m_fp_bing))
recall_g_f_bing=(count_g_f_tp_bing/(count_g_f_tp_bing+count_g_f_fp_bing))

#F1-Score
f1_score_g_m_bing=(2*(precision_g_m_bing*recall_g_m_bing)/(precision_g_m_bing+recall_g_m_bing))
f1_score_g_f_bing=(2*(precision_g_f_bing*recall_g_f_bing)/(precision_g_f_bing+recall_g_f_bing))

#G Wert
G_m_f_bing=(f1_score_g_m_bing-f1_score_g_f_bing)
print(f"G-Wert: {G_m_f_bing:.2f} ")

Acc-Wert: 51.56% 
S-Wert: 0.12 
G-Wert: 0.35 


In [10]:
#Deepl

In [11]:
#2. Dateianalyse
# Datei einlesen
with open('deepl.csv', 'r', encoding='utf-8') as file:
    text_deepl_satz = file.read()

# Text analysieren
doc_deepl_satz = nlp(text_deepl_satz)

# Liste für die Tabellenzeilen erstellen
data_deepl_satz = []

for i, sentence in enumerate(doc_deepl_satz.sentences, start=1):
    for word in sentence.words:
        data_deepl_satz.append({
            "Satz": i,#satz
            "Deepl Bezeichnung (Satz)": word.text,
            "Deepl Geschlecht (Satz)": word.feats,
            "Position": word.id,
            "xpos":word.xpos

        })
        
def extract_gender(text_deepl_satz):
    if text_deepl_satz:  # Prüft, ob der Wert nicht None oder leer ist
        if "Gender=Fem" in text_deepl_satz:
            return "female"
        elif "Gender=Masc" in text_deepl_satz:
            return "male"
    return "unknown"


# Erstelle den DataFrame
df_deepl_satz = pd.DataFrame(data_deepl_satz)
df_deepl_satz['Index'] = df_deepl_satz.index
df_deepl_satz["Deepl Geschlecht (Satz)"] = df_deepl_satz["Deepl Geschlecht (Satz)"].apply(extract_gender)

# Ausgabe des DataFrames
#print(df_deepl_satz)
df_deepl_satz.head()

,Satz,Deepl Bezeichnung (Satz),Deepl Geschlecht (Satz),Position,xpos,Index
0,1,Der,male,1,ART,0
1,1,Techniker,male,2,NN,1
2,1,sagte,unknown,3,VVFIN,2
3,1,dem,male,4,ART,3
4,1,Kunden,male,5,NN,4


In [12]:
#3. Zusammenfügen der Daten
#filtern berufe + pronom
df_p0_deepl = df_deepl_satz.merge(df_morph_satz, on=['Satz'])
df_p0_deepl = df_p0_deepl.loc[df_p0_deepl['xpos'] == 'NN']

df_p1_deepl = df_p0_deepl[df_p0_deepl['Position_en'] == 2 ].groupby('Satz').first()
df_p2_deepl = df_p0_deepl[(df_p0_deepl['Position_en'] > 2) & (df_p0_deepl['Position'] != 2)].groupby('Satz').first()

df_p_deepl=  pd.concat([df_p1_deepl, df_p2_deepl]).sort_values('Index')

df_p_deepl["Index"] = range(len(df_p_deepl))


#statistik spalte vereinen mit df_gender (Gender: male, female, neutral // n/ST//index) 
#index ->  n/ST Spalte
df_result_deepl_satz = df_p_deepl.merge(df_gender, on=['Index'])
df_result_deepl_satz["Acc"] = np.where(df_result_deepl_satz["Deepl Geschlecht (Satz)"] == df_result_deepl_satz["Soll-Geschlecht"], "ja", "nein")
#Spalte, die winogender Sätze nach stereotypisch und anti-stereotypisch einordnet
df_result_deepl_satz['Soll (n/ST)'] = np.where((df_result_deepl_satz['Soll-Geschlecht'] == 'neutral')  |
                                             (df_result_deepl_satz['Soll-Geschlecht'] == 'unbekannt'),'-',                                              
                                            np.where (((df_result_deepl_satz['Soll-Geschlecht'] == 'male')  &
                                               (df_result_deepl_satz['n/ST'] < 50)) |
                                                ((df_result_deepl_satz['Soll-Geschlecht'] == 'female') &
                                                  (df_result_deepl_satz['n/ST'] > 50)),'ST', 'n/ST'))
#Spalte n/ST der Übersetzungen
df_result_deepl_satz['Ist (n/ST)'] = np.where((df_result_deepl_satz['Deepl Geschlecht (Satz)'] == 'neutral') |
                                    (df_result_deepl_satz['Deepl Geschlecht (Satz)'] == 'unbekannt'),'-',
                                             
                                    np.where(((df_result_deepl_satz['Deepl Geschlecht (Satz)'] == 'male') & 
                                     (df_result_deepl_satz['n/ST'] < 50)) |
                                    ((df_result_deepl_satz['Deepl Geschlecht (Satz)'] == 'female') & 
                                     (df_result_deepl_satz['n/ST'] > 50)),'ST', 'n/ST'))
df_result_deepl_satz.head()

,Deepl Bezeichnung (Satz),Deepl Geschlecht (Satz),Position,xpos,Index,Token Text,Position_en,Index_en,Soll-Geschlecht,n/ST,Acc,Soll (n/ST),Ist (n/ST)
0,Techniker,male,2,NN,0,$OCCUPATION,2,0,male,40.34,ja,ST,ST
1,Techniker,male,2,NN,1,$OCCUPATION,2,1,female,40.34,nein,n/ST,ST
2,Techniker,male,2,NN,2,$OCCUPATION,2,2,neutral,40.34,nein,-,ST
3,Techniker,male,2,NN,3,$OCCUPATION,2,3,male,40.34,ja,ST,ST
4,Techniker,male,2,NN,4,$OCCUPATION,2,4,female,40.34,nein,n/ST,ST


In [13]:
#4. Auswerten für Berechnung

#M:F wieviele Ergebnisse männlich/weiblich: zahlen + prozent
total = len(df_result_deepl_satz)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_deepl_satz = df_result_deepl_satz["Deepl Geschlecht (Satz)"].value_counts().get("male", 0)
count_fem_deepl_satz = df_result_deepl_satz["Deepl Geschlecht (Satz)"].value_counts().get("female", 0)
count_un_deepl_satz = df_result_deepl_satz["Deepl Geschlecht (Satz)"].value_counts().get("Unbekannt", 0)
count_neut_deepl_satz = df_result_deepl_satz["Deepl Bezeichnung (Satz)"].value_counts().get("neutral", 0)

# Prozentsätze berechnen
percent_masc_deepl_satz = (count_masc_deepl_satz / total) * 100 if total > 0 else 0
percent_fem_deepl_satz = (count_fem_deepl_satz / total) * 100 if total > 0 else 0
percent_un_deepl_satz = (count_un_deepl_satz / total) * 100 if total > 0 else 0
percent_neut_deepl_satz = (count_neut_deepl_satz / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (male): {count_masc_deepl_satz} ({percent_masc_deepl_satz:.2f}%)")
print(f"Weiblich (female): {count_fem_deepl_satz} ({percent_fem_deepl_satz:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_deepl_satz} ({percent_un_deepl_satz:.2f}%)")
print(f"Neutral (neutral): {count_neut_deepl_satz} ({percent_neut_deepl_satz:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_result_deepl_satz[df_result_deepl_satz["Deepl Geschlecht (Satz)"] == "male"]
fem_rows = df_result_deepl_satz[df_result_deepl_satz["Deepl Geschlecht (Satz)"] == "female"]
row_deepl_satz=len(df_result_deepl_satz.index)

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
under_50_m_deepl_satz = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_deepl_satz = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_deepl_satz = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_deepl_satz = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_deepl_satz} von {row_deepl_satz} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_deepl_satz} von {row_deepl_satz} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_deepl_satz} von {row_deepl_satz} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_deepl_satz} von {row_deepl_satz} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (male): 541 (75.14%)
Weiblich (female): 169 (23.47%)
Unbekannt (Unbekannt): 0 (0.00%)
Neutral (neutral): 0 (0.00%)

302 von 720 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
239 von 720 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
36 von 720 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
133 von 720 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [14]:
#5. Auswertung (Acc, S, G)
#Acc berechenen wo oft das Geschlecht (m/f) richtig ist (Berechnung nach 1. Studie)


count_acc_male = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Acc"] == "ja") & (df_result_deepl_satz["Soll-Geschlecht"] == "male"), "Soll-Geschlecht"].count()

count_acc_female = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Acc"] == "ja") & (df_result_deepl_satz["Soll-Geschlecht"] == "female"), "Soll-Geschlecht"].count()
#Menge an korrekt ermittelten geschlechtlichen Werten
count_m_f_deepl_satz = count_acc_male + count_acc_female
#Prozent ermitteln
prozent_acc_deepl= count_m_f_deepl_satz/(count_masc_deepl_satz+count_fem_deepl_satz) * 100 if total > 0 else 0

print(f"Acc-Wert: {prozent_acc_deepl:.2f}% ")

# S berechnen mit F1-Score (2x f1-score berechnen: n/ST und ST)
#TP, FP, FN
#TP
count_s_st_tp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="ST") & (df_result_deepl_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FP
count_s_st_fp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="ST") & (df_result_deepl_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FN
count_s_st_fn_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="n/ST") & (df_result_deepl_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#TP
count_s_nst_tp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="n/ST") & (df_result_deepl_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FP
count_s_nst_fp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="n/ST") & (df_result_deepl_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FN
count_s_nst_fn_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll (n/ST)"] =="ST") & (df_result_deepl_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()

#Precision
#ST
precision_s_st_deepl= (count_s_st_tp_deepl/(count_s_st_tp_deepl+count_s_st_fp_deepl))
#nST
precision_s_nst_deepl= (count_s_nst_tp_deepl/(count_s_nst_tp_deepl+count_s_nst_fp_deepl))

#Recall
#ST
recall_s_st_deepl= (count_s_st_tp_deepl/(count_s_st_tp_deepl+count_s_st_fn_deepl))
#nST
recall_s_nst_deepl=(count_s_nst_tp_deepl/(count_s_nst_tp_deepl+count_s_nst_fn_deepl))

#F1-Score
#ST
f1_score_s_st_deepl=(2*(precision_s_st_deepl*recall_s_st_deepl)/(precision_s_st_deepl+recall_s_st_deepl))
#nST
f1_score_s_nst_deepl=(2*(precision_s_nst_deepl*recall_s_nst_deepl)/(precision_s_nst_deepl+recall_s_nst_deepl))
# n/ST und ST F1-Score berechnen => S
S_st_nst_deepl=(f1_score_s_st_deepl-f1_score_s_nst_deepl)
print(f"S-Wert: {S_st_nst_deepl:.2f} ")


#G berechnen mit F1-Score
#TP, FP, FN
#TP
count_g_m_tp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="male") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FP
count_g_m_fp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="male") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"] == "female"), "Soll-Geschlecht"].count()

#FN
count_g_m_fn_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="female") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"]== "female"), "Soll-Geschlecht"].count()

#TP
count_g_f_tp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="female") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"]== "female"),"Soll-Geschlecht"].count()

#FP
count_g_f_fp_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="female") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FN
count_g_f_fn_deepl = df_result_deepl_satz.loc[
    (df_result_deepl_satz["Soll-Geschlecht"] =="male") & (df_result_deepl_satz["Deepl Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#Precision
precision_g_m_deepl=(count_g_m_tp_deepl/(count_g_m_tp_deepl+count_g_m_fp_deepl))
precision_g_f_deepl=(count_g_f_tp_deepl/(count_g_f_tp_deepl+count_g_f_fp_deepl))

#Recall
recall_g_m_deepl=(count_g_m_tp_deepl/(count_g_m_tp_deepl+count_g_m_fp_deepl))
recall_g_f_deepl=(count_g_f_tp_deepl/(count_g_f_tp_deepl+count_g_f_fp_deepl))

#F1-Score
f1_score_g_m_deepl=(2*(precision_g_m_deepl*recall_g_m_deepl)/(precision_g_m_deepl+recall_g_m_deepl))
f1_score_g_f_deepl=(2*(precision_g_f_deepl*recall_g_f_deepl)/(precision_g_f_deepl+recall_g_f_deepl))

#G Wert
G_m_f_deepl=(f1_score_g_m_deepl-f1_score_g_f_deepl)
print(f"G-Wert: {G_m_f_deepl:.2f} ")


Acc-Wert: 37.89% 
S-Wert: 0.18 
G-Wert: 0.47 


In [15]:
#Google

In [16]:
#2. Dateianalyse
# Datei einlesen
with open('google.csv', 'r', encoding='utf-8') as file:
    text_google_satz = file.read()

# Text analysieren
doc_google_satz = nlp(text_google_satz)

# Liste für die Tabellenzeilen erstellen
data_google_satz = []

for i, sentence in enumerate(doc_google_satz.sentences, start=1):
    for word in sentence.words:
        data_google_satz.append({
            "Satz": i,#satz
            "Google Bezeichnung (Satz)": word.text,
            "Google Geschlecht (Satz)": word.feats,
            "Position": word.id,
            "xpos":word.xpos

        })
        
def extract_gender(text_google_satz):
    if text_google_satz:  # Prüft, ob der Wert nicht None oder leer ist
        if "Gender=Fem" in text_google_satz:
            return "female"
        elif "Gender=Masc" in text_google_satz:
            return "male"
    return "unknown"


# Erstelle den DataFrame
df_google_satz = pd.DataFrame(data_google_satz)
df_google_satz['Index'] = df_google_satz.index
df_google_satz["Google Geschlecht (Satz)"] = df_google_satz["Google Geschlecht (Satz)"].apply(extract_gender)

# Ausgabe des DataFrames
#print(df_google_satz)
df_google_satz.head()


,Satz,Google Bezeichnung (Satz),Google Geschlecht (Satz),Position,xpos,Index
0,1,Der,male,1,ART,0
1,1,Techniker,male,2,NN,1
2,1,teilte,unknown,3,VVFIN,2
3,1,dem,male,4,ART,3
4,1,Kunden,male,5,NN,4


In [17]:
#3. Zusammenfügen der Daten
#filtern berufe + pronom
df_p0_google = df_google_satz.merge(df_morph_satz, on=['Satz'])
df_p0_google = df_p0_google.loc[df_p0_google['xpos'] == 'NN']

df_p1_google = df_p0_google[df_p0_google['Position_en'] == 2 ].groupby('Satz').first()
df_p2_google = df_p0_google[(df_p0_google['Position_en'] > 2) & (df_p0_google['Position'] != 2)].groupby('Satz').first()

df_p_google=  pd.concat([df_p1_google, df_p2_google]).sort_values('Index')

df_p_google["Index"] = range(len(df_p_google))

#statistik spalte vereinen mit df_gender (Gender: male, female, neutral // n/ST//index) 
#index ->  n/ST Spalte
df_result_google_satz = df_p_google.merge(df_gender, on=['Index'])
df_result_google_satz["Acc"] = np.where(df_result_google_satz["Google Geschlecht (Satz)"] == df_result_google_satz["Soll-Geschlecht"], "ja", "nein")
#Spalte, die winogender Sätze nach stereotypisch und anti-stereotypisch einordnet
df_result_google_satz['Soll (n/ST)'] = np.where((df_result_google_satz['Soll-Geschlecht'] == 'neutral')  |
                                             (df_result_google_satz['Soll-Geschlecht'] == 'unbekannt'),'-',                                              
                                            np.where (((df_result_google_satz['Soll-Geschlecht'] == 'male')  &
                                               (df_result_google_satz['n/ST'] < 50)) |
                                                ((df_result_google_satz['Soll-Geschlecht'] == 'female') &
                                                  (df_result_google_satz['n/ST'] > 50)),'ST', 'n/ST'))
#Spalte n/ST der Übersetzungen
df_result_google_satz['Ist (n/ST)'] = np.where((df_result_google_satz['Google Geschlecht (Satz)'] == 'neutral') |
                                    (df_result_google_satz['Google Geschlecht (Satz)'] == 'unbekannt'),'-',
                                             
                                    np.where(((df_result_google_satz['Google Geschlecht (Satz)'] == 'male') & 
                                     (df_result_google_satz['n/ST'] < 50)) |
                                    ((df_result_google_satz['Google Geschlecht (Satz)'] == 'female') & 
                                     (df_result_google_satz['n/ST'] > 50)),'ST', 'n/ST'))
df_result_google_satz.head()


,Google Bezeichnung (Satz),Google Geschlecht (Satz),Position,xpos,Index,Token Text,Position_en,Index_en,Soll-Geschlecht,n/ST,Acc,Soll (n/ST),Ist (n/ST)
0,Techniker,male,2,NN,0,$OCCUPATION,2,0,male,40.34,ja,ST,ST
1,Techniker,male,2,NN,1,$OCCUPATION,2,1,female,40.34,nein,n/ST,ST
2,Techniker,male,2,NN,2,$OCCUPATION,2,2,neutral,40.34,nein,-,ST
3,Techniker,male,2,NN,3,$OCCUPATION,2,3,male,40.34,ja,ST,ST
4,Techniker,male,2,NN,4,$OCCUPATION,2,4,female,40.34,nein,n/ST,ST


In [18]:
#4. Auswerten für Berechnung

#M:F wieviele Ergebnisse männlich/weiblich: zahlen + prozent
total = len(df_result_google_satz)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_google_satz = df_result_google_satz["Google Geschlecht (Satz)"].value_counts().get("male", 0)
count_fem_google_satz = df_result_google_satz["Google Geschlecht (Satz)"].value_counts().get("female", 0)
count_un_google_satz = df_result_google_satz["Google Geschlecht (Satz)"].value_counts().get("Unbekannt", 0)
count_neut_google_satz = df_result_google_satz["Google Bezeichnung (Satz)"].value_counts().get("neutral", 0)

# Prozentsätze berechnen
percent_masc_google_satz = (count_masc_google_satz / total) * 100 if total > 0 else 0
percent_fem_google_satz = (count_fem_google_satz / total) * 100 if total > 0 else 0
percent_un_google_satz = (count_un_google_satz / total) * 100 if total > 0 else 0
percent_neut_google_satz = (count_neut_google_satz / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (male): {count_masc_google_satz} ({percent_masc_google_satz:.2f}%)")
print(f"Weiblich (female): {count_fem_google_satz} ({percent_fem_google_satz:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_google_satz} ({percent_un_google_satz:.2f}%)")
print(f"Neutral (neutral): {count_neut_google_satz} ({percent_neut_google_satz:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_result_google_satz[df_result_google_satz["Google Geschlecht (Satz)"] == "male"]
fem_rows = df_result_google_satz[df_result_google_satz["Google Geschlecht (Satz)"] == "female"]
row_google_satz=len(df_result_google_satz.index)

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
under_50_m_google_satz = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_google_satz = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_google_satz = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_google_satz = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_google_satz} von {row_google_satz} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_google_satz} von {row_google_satz} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_google_satz} von {row_google_satz} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_google_satz} von {row_google_satz} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (male): 489 (67.92%)
Weiblich (female): 214 (29.72%)
Unbekannt (Unbekannt): 0 (0.00%)
Neutral (neutral): 0 (0.00%)

264 von 720 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
225 von 720 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
69 von 720 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
145 von 720 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [20]:
#5. Auswertung (Acc, S, G)
#Acc berechenen wo oft das Geschlecht (m/f) richtig ist (Berechnung nach 1. Studie)


count_acc_male = df_result_google_satz.loc[
    (df_result_google_satz["Acc"] == "ja") & (df_result_google_satz["Soll-Geschlecht"] == "male"), "Soll-Geschlecht"].count()

count_acc_female = df_result_google_satz.loc[
    (df_result_google_satz["Acc"] == "ja") & (df_result_google_satz["Soll-Geschlecht"] == "female"), "Soll-Geschlecht"].count()
#Menge an korrekt ermittelten geschlechtlichen Werten
count_m_f_google_satz = count_acc_male + count_acc_female
#Prozent ermitteln
prozent_acc_google= count_m_f_google_satz/(count_masc_google_satz+count_fem_google_satz) * 100 if total > 0 else 0

print(f"Acc-Wert: {prozent_acc_google:.2f}% ")

# S berechnen mit F1-Score (2x f1-score berechnen: n/ST und ST)
#TP, FP, FN
#TP
count_s_st_tp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="ST") & (df_result_google_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FP
count_s_st_fp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="ST") & (df_result_google_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FN
count_s_st_fn_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="n/ST") & (df_result_google_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#TP
count_s_nst_tp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="n/ST") & (df_result_google_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FP
count_s_nst_fp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="n/ST") & (df_result_google_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FN
count_s_nst_fn_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll (n/ST)"] =="ST") & (df_result_google_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()

#Precision
#ST
precision_s_st_google= (count_s_st_tp_google/(count_s_st_tp_google+count_s_st_fp_google))
#nST
precision_s_nst_google= (count_s_nst_tp_google/(count_s_nst_tp_google+count_s_nst_fp_google))

#Recall
#ST
recall_s_st_google= (count_s_st_tp_google/(count_s_st_tp_google+count_s_st_fn_google))
#nST
recall_s_nst_google=(count_s_nst_tp_google/(count_s_nst_tp_google+count_s_nst_fn_google))

#F1-Score
#ST
f1_score_s_st_google=(2*(precision_s_st_google*recall_s_st_google)/(precision_s_st_google+recall_s_st_google))
#nST
f1_score_s_nst_google=(2*(precision_s_nst_google*recall_s_nst_google)/(precision_s_nst_google+recall_s_nst_google))
# n/ST und ST F1-Score berechnen => S
S_st_nst_google=(f1_score_s_st_google-f1_score_s_nst_google)
print(f"S-Wert: {S_st_nst_google:.2f} ")


#G berechnen mit F1-Score
#TP, FP, FN
#TP
count_g_m_tp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="male") & (df_result_google_satz["Google Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FP
count_g_m_fp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="male") & (df_result_google_satz["Google Geschlecht (Satz)"] == "female"), "Soll-Geschlecht"].count()

#FN
count_g_m_fn_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="female") & (df_result_google_satz["Google Geschlecht (Satz)"]== "female"), "Soll-Geschlecht"].count()

#TP
count_g_f_tp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="female") & (df_result_google_satz["Google Geschlecht (Satz)"]== "female"),"Soll-Geschlecht"].count()

#FP
count_g_f_fp_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="female") & (df_result_google_satz["Google Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FN
count_g_f_fn_google = df_result_google_satz.loc[
    (df_result_google_satz["Soll-Geschlecht"] =="male") & (df_result_google_satz["Google Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#Precision
precision_g_m_google=(count_g_m_tp_google/(count_g_m_tp_google+count_g_m_fp_google))
precision_g_f_google=(count_g_f_tp_google/(count_g_f_tp_google+count_g_f_fp_google))

#Recall
recall_g_m_google=(count_g_m_tp_google/(count_g_m_tp_google+count_g_m_fp_google))
recall_g_f_google=(count_g_f_tp_google/(count_g_f_tp_google+count_g_f_fp_google))

#F1-Score
f1_score_g_m_google=(2*(precision_g_m_google*recall_g_m_google)/(precision_g_m_google+recall_g_m_google))
f1_score_g_f_google=(2*(precision_g_f_google*recall_g_f_google)/(precision_g_f_google+recall_g_f_google))

#G Wert
G_m_f_google=(f1_score_g_m_google-f1_score_g_f_google)#*100 if total > 0 else 0
print(f"G-Wert: {G_m_f_google:.2f} ")


Acc-Wert: 41.39% 
S-Wert: 0.11 
G-Wert: 0.33 


In [21]:
#GPT

In [22]:
#2. Dateianalyse
# Datei einlesen
with open('gpt-4o_mini.csv', 'r', encoding='utf-8') as file:
    text_gpt_satz = file.read()

# Text analysieren
doc_gpt_satz = nlp(text_gpt_satz)

# Liste für die Tabellenzeilen erstellen
data_gpt_satz = []

for i, sentence in enumerate(doc_gpt_satz.sentences, start=1):
    for word in sentence.words:
        data_gpt_satz.append({
            "Satz": i,#satz
            "GPT Bezeichnung (Satz)": word.text,
            "GPT Geschlecht (Satz)": word.feats,
            "Position": word.id,
            "xpos":word.xpos

        })
        
def extract_gender(text_gpt_satz):
    if text_gpt_satz:  # Prüft, ob der Wert nicht None oder leer ist
        if "Gender=Fem" in text_gpt_satz:
            return "female"
        elif "Gender=Masc" in text_gpt_satz:
            return "male"
    return "unknown"


# Erstelle den DataFrame
df_gpt_satz = pd.DataFrame(data_gpt_satz)
df_gpt_satz['Index'] = df_gpt_satz.index
df_gpt_satz["GPT Geschlecht (Satz)"] = df_gpt_satz["GPT Geschlecht (Satz)"].apply(extract_gender)

# Ausgabe des DataFrames
#print(df_gpt_satz)
df_gpt_satz.head()


,Satz,GPT Bezeichnung (Satz),GPT Geschlecht (Satz),Position,xpos,Index
0,1,Der,male,1,ART,0
1,1,Techniker,male,2,NN,1
2,1,sagte,unknown,3,VVFIN,2
3,1,dem,male,4,ART,3
4,1,Kunden,male,5,NN,4


In [24]:
#3. Zusammenfügen der Daten
#filtern berufe + pronom
df_p0_gpt = df_gpt_satz.merge(df_morph_satz, on=['Satz'])
df_p0_gpt = df_p0_gpt.loc[df_p0_gpt['xpos'] == 'NN']

df_p1_gpt = df_p0_gpt[df_p0_gpt['Position_en'] == 2 ].groupby('Satz').first()
df_p2_gpt = df_p0_gpt[(df_p0_gpt['Position_en'] > 2) & (df_p0_gpt['Position'] != 2)].groupby('Satz').first()

df_p_gpt=  pd.concat([df_p1_gpt, df_p2_gpt]).sort_values('Index')

df_p_gpt["Index"] = range(len(df_p_gpt))


#statistik spalte vereinen mit df_gender (Gender: male, female, neutral // n/ST//index) 
#index ->  n/ST Spalte
df_result_gpt_satz = df_p_gpt.merge(df_gender, on=['Index'])
df_result_gpt_satz["Acc"] = np.where(df_result_gpt_satz["GPT Geschlecht (Satz)"] == df_result_gpt_satz["Soll-Geschlecht"], "ja", "nein")
#Spalte, die winogender Sätze nach stereotypisch und anti-stereotypisch einordnet
df_result_gpt_satz['Soll (n/ST)'] = np.where((df_result_gpt_satz['Soll-Geschlecht'] == 'neutral')  |
                                             (df_result_gpt_satz['Soll-Geschlecht'] == 'unbekannt'),'-',                                              
                                            np.where (((df_result_gpt_satz['Soll-Geschlecht'] == 'male')  &
                                               (df_result_gpt_satz['n/ST'] < 50)) |
                                                ((df_result_gpt_satz['Soll-Geschlecht'] == 'female') &
                                                  (df_result_gpt_satz['n/ST'] > 50)),'ST', 'n/ST'))
#Spalte n/ST der Übersetzungen
df_result_gpt_satz['Ist (n/ST)'] = np.where((df_result_gpt_satz['GPT Geschlecht (Satz)'] == 'neutral') |
                                    (df_result_gpt_satz['GPT Geschlecht (Satz)'] == 'unbekannt'),'-',
                                             
                                    np.where(((df_result_gpt_satz['GPT Geschlecht (Satz)'] == 'male') & 
                                     (df_result_gpt_satz['n/ST'] < 50)) |
                                    ((df_result_gpt_satz['GPT Geschlecht (Satz)'] == 'female') & 
                                     (df_result_gpt_satz['n/ST'] > 50)),'ST', 'n/ST'))
df_result_gpt_satz.head()


,GPT Bezeichnung (Satz),GPT Geschlecht (Satz),Position,xpos,Index,Token Text,Position_en,Index_en,Soll-Geschlecht,n/ST,Acc,Soll (n/ST),Ist (n/ST)
0,Techniker,male,2,NN,0,$OCCUPATION,2,0,male,40.34,ja,ST,ST
1,Techniker,male,2,NN,1,$OCCUPATION,2,1,female,40.34,nein,n/ST,ST
2,Techniker,male,2,NN,2,$OCCUPATION,2,2,neutral,40.34,nein,-,ST
3,Techniker,male,2,NN,3,$OCCUPATION,2,3,male,40.34,ja,ST,ST
4,Techniker,male,2,NN,4,$OCCUPATION,2,4,female,40.34,nein,n/ST,ST


In [25]:
#4. Auswerten für Berechnung

#M:F wieviele Ergebnisse männlich/weiblich: zahlen + prozent
total = len(df_result_gpt_satz)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_gpt_satz = df_result_gpt_satz["GPT Geschlecht (Satz)"].value_counts().get("male", 0)
count_fem_gpt_satz = df_result_gpt_satz["GPT Geschlecht (Satz)"].value_counts().get("female", 0)
count_un_gpt_satz = df_result_gpt_satz["GPT Geschlecht (Satz)"].value_counts().get("Unbekannt", 0)
count_neut_gpt_satz = df_result_gpt_satz["GPT Bezeichnung (Satz)"].value_counts().get("neutral", 0)

# Prozentsätze berechnen
percent_masc_gpt_satz = (count_masc_gpt_satz / total) * 100 if total > 0 else 0
percent_fem_gpt_satz = (count_fem_gpt_satz / total) * 100 if total > 0 else 0
percent_un_gpt_satz = (count_un_gpt_satz / total) * 100 if total > 0 else 0
percent_neut_gpt_satz = (count_neut_gpt_satz / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (male): {count_masc_gpt_satz} ({percent_masc_gpt_satz:.2f}%)")
print(f"Weiblich (female): {count_fem_gpt_satz} ({percent_fem_gpt_satz:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_gpt_satz} ({percent_un_gpt_satz:.2f}%)")
print(f"Neutral (neutral): {count_neut_gpt_satz} ({percent_neut_gpt_satz:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_result_gpt_satz[df_result_gpt_satz["GPT Geschlecht (Satz)"] == "male"]
fem_rows = df_result_gpt_satz[df_result_gpt_satz["GPT Geschlecht (Satz)"] == "female"]
row_gpt_satz=len(df_result_gpt_satz.index)

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
under_50_m_gpt_satz = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_gpt_satz = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_gpt_satz = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_gpt_satz = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_gpt_satz} von {row_gpt_satz} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_gpt_satz} von {row_gpt_satz} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_gpt_satz} von {row_gpt_satz} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_gpt_satz} von {row_gpt_satz} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (male): 617 (85.69%)
Weiblich (female): 91 (12.64%)
Unbekannt (Unbekannt): 0 (0.00%)
Neutral (neutral): 0 (0.00%)

324 von 720 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
293 von 720 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
24 von 720 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
67 von 720 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [27]:
#5. Auswertung (Acc, S, G)
#Acc berechenen wo oft das Geschlecht (m/f) richtig ist (Berechnung nach 1. Studie)


count_acc_male = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Acc"] == "ja") & (df_result_gpt_satz["Soll-Geschlecht"] == "male"), "Soll-Geschlecht"].count()

count_acc_female = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Acc"] == "ja") & (df_result_gpt_satz["Soll-Geschlecht"] == "female"), "Soll-Geschlecht"].count()
#Menge an korrekt ermittelten geschlechtlichen Werten
count_m_f_gpt_satz = count_acc_male + count_acc_female
#Prozent ermitteln
prozent_acc_gpt= count_m_f_gpt_satz/(count_masc_gpt_satz+count_fem_gpt_satz) * 100 if total > 0 else 0

print(f"Acc-Wert: {prozent_acc_gpt:.2f}% ")

# S berechnen mit F1-Score (2x f1-score berechnen: n/ST und ST)
#TP, FP, FN
#TP
count_s_st_tp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="ST") & (df_result_gpt_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FP
count_s_st_fp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="ST") & (df_result_gpt_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FN
count_s_st_fn_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="n/ST") & (df_result_gpt_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#TP
count_s_nst_tp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="n/ST") & (df_result_gpt_satz['Ist (n/ST)'] == "n/ST"), "Soll (n/ST)"].count()
#FP
count_s_nst_fp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="n/ST") & (df_result_gpt_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()
#FN
count_s_nst_fn_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll (n/ST)"] =="ST") & (df_result_gpt_satz['Ist (n/ST)'] == "ST"), "Soll (n/ST)"].count()

#Precision
#ST
precision_s_st_gpt= (count_s_st_tp_gpt/(count_s_st_tp_gpt+count_s_st_fp_gpt))
#nST
precision_s_nst_gpt= (count_s_nst_tp_gpt/(count_s_nst_tp_gpt+count_s_nst_fp_gpt))

#Recall
#ST
recall_s_st_gpt= (count_s_st_tp_gpt/(count_s_st_tp_gpt+count_s_st_fn_gpt))
#nST
recall_s_nst_gpt=(count_s_nst_tp_gpt/(count_s_nst_tp_gpt+count_s_nst_fn_gpt))

#F1-Score
#ST
f1_score_s_st_gpt=(2*(precision_s_st_gpt*recall_s_st_gpt)/(precision_s_st_gpt+recall_s_st_gpt))
#nST
f1_score_s_nst_gpt=(2*(precision_s_nst_gpt*recall_s_nst_gpt)/(precision_s_nst_gpt+recall_s_nst_gpt))
# n/ST und ST F1-Score berechnen => S
S_st_nst_gpt=(f1_score_s_st_gpt-f1_score_s_nst_gpt)
print(f"S-Wert: {S_st_nst_gpt:.2f} ")


#G berechnen mit F1-Score
#TP, FP, FN
#TP
count_g_m_tp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="male") & (df_result_gpt_satz["GPT Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FP
count_g_m_fp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="male") & (df_result_gpt_satz["GPT Geschlecht (Satz)"] == "female"), "Soll-Geschlecht"].count()

#FN
count_g_m_fn_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="female") & (df_result_gpt_satz["GPT Geschlecht (Satz)"]== "female"), "Soll-Geschlecht"].count()

#TP
count_g_f_tp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="female") & (df_result_gpt_satz["GPT Geschlecht (Satz)"]== "female"),"Soll-Geschlecht"].count()

#FP
count_g_f_fp_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="female") & (df_result_gpt_satz["GPT Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#FN
count_g_f_fn_gpt = df_result_gpt_satz.loc[
    (df_result_gpt_satz["Soll-Geschlecht"] =="male") & (df_result_gpt_satz["GPT Geschlecht (Satz)"]== "male"), "Soll-Geschlecht"].count()

#Precision
precision_g_m_gpt=(count_g_m_tp_gpt/(count_g_m_tp_gpt+count_g_m_fp_gpt))
precision_g_f_gpt=(count_g_f_tp_gpt/(count_g_f_tp_gpt+count_g_f_fp_gpt))

#Recall
recall_g_m_gpt=(count_g_m_tp_gpt/(count_g_m_tp_gpt+count_g_m_fp_gpt))
recall_g_f_gpt=(count_g_f_tp_gpt/(count_g_f_tp_gpt+count_g_f_fp_gpt))

#F1-Score
f1_score_g_m_gpt=(2*(precision_g_m_gpt*recall_g_m_gpt)/(precision_g_m_gpt+recall_g_m_gpt))
f1_score_g_f_gpt=(2*(precision_g_f_gpt*recall_g_f_gpt)/(precision_g_f_gpt+recall_g_f_gpt))

#G Wert
G_m_f_gpt=(f1_score_g_m_gpt-f1_score_g_f_gpt)
print(f"G-Wert: {G_m_f_gpt:.2f} ")


Acc-Wert: 37.71% 
S-Wert: 0.08 
G-Wert: 0.70 


In [28]:
#Für Grafiken Variabeln speichern mit store%

In [29]:
#Jemand traf sich mit dem Buchhalter, um herauszufinden, ob er bei der Steuervorbereitung helfen könnte.

doc = nlp("Der Techniker teilte der Kundin mit, dass sie mit Bargeld bezahlen könne. Jemand traf sich mit dem Buchhalter, um herauszufinden, ob er bei der Steuervorbereitung helfen könnte.")
#print(doc)

In [30]:
#Bing
%store prozent_acc_bing
%store S_st_nst_bing
%store G_m_f_bing
#Deepl
%store prozent_acc_deepl
%store S_st_nst_deepl
%store G_m_f_deepl
#Google
%store prozent_acc_google
%store S_st_nst_google
%store G_m_f_google
#GPT
%store prozent_acc_gpt
%store S_st_nst_gpt
%store G_m_f_gpt

Stored 'prozent_acc_bing' (float64)
Stored 'S_st_nst_bing' (float64)
Stored 'G_m_f_bing' (float64)
Stored 'prozent_acc_deepl' (float64)
Stored 'S_st_nst_deepl' (float64)
Stored 'G_m_f_deepl' (float64)
Stored 'prozent_acc_google' (float64)
Stored 'S_st_nst_google' (float64)
Stored 'G_m_f_google' (float64)
Stored 'prozent_acc_gpt' (float64)
Stored 'S_st_nst_gpt' (float64)
Stored 'G_m_f_gpt' (float64)
